<a href="https://colab.research.google.com/github/ruangroc/KMeansCitySearchTool/blob/main/City_Search_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# City Search Tool

#### Datasets Used
- [MoveHub City Ratings](https://www.kaggle.com/blitzr/movehub-city-rankings?select=movehubqualityoflife.csv)
- [colleges and universities in the US](https://hifld-geoplatform.opendata.arcgis.com/datasets/colleges-and-universities/data)



In [17]:
import pandas as pd
df = pd.read_csv('https://drive.google.com/uc?id=1jy6POTqn8XQL-azM7j4tcYPyKSAlSZdy')
cost_of_living = pd.read_csv('https://drive.google.com/uc?id=1BBs0qKCPBSQbsPW3I_3NegFAxXCeYYpn')
colleges = pd.read_csv('https://drive.google.com/uc?id=1OqrQ8J85rkuEHGOJw691vp_2yG1ySkNA')

In [18]:
df.head()

,City,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,lat,lng
0,Caracas,65.18,11.25,44.44,83.45,8.61,85.70,10.480594,-66.903606
1,Johannesburg,84.08,53.99,59.98,47.39,51.26,83.93,-26.204103,28.047305
2,Fortaleza,80.17,52.28,45.46,66.32,36.68,78.65,-3.732714,-38.526998
3,Saint Louis,85.25,80.40,77.29,31.33,87.51,78.13,38.627003,-90.199404
4,Mexico City,75.07,24.28,61.76,18.95,27.91,77.86,19.432608,-99.133208


In [19]:
cost_of_living.head()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income
0,Lausanne,3.15,12.59,8.40,1.32,1714.00,4266.11
1,Zurich,3.28,12.59,8.40,1.31,2378.61,4197.55
2,Geneva,2.80,12.94,10.49,1.28,2607.95,3917.72
3,Basel,3.50,11.89,7.35,1.25,1649.29,3847.76
4,Perth,2.87,11.43,10.08,0.97,2083.14,3358.55


In [20]:
# colleges.head()
colleges_usa = colleges.loc[colleges['COUNTRY'] == 'USA']
colleges_usa_small = colleges_usa.drop(columns=['X','Y','OBJECTID','IPEDSID','ADDRESS','STATE','ZIP','ZIP4','TELEPHONE',
                                  'TYPE','STATUS','POPULATION','COUNTY','COUNTYFIPS','COUNTRY','LATITUDE',
                                  'LONGITUDE','NAICS_CODE','NAICS_DESC','SOURCE','SOURCEDATE','VAL_METHOD',
                                  'WEBSITE','STFIPS','COFIPS','SECTOR','LEVEL_','HI_OFFER','DEG_GRANT',
                                  'LOCALE','CLOSE_DATE', 'MERGE_ID', 'ALIAS', 'SIZE_SET', 'INST_SIZE', 
                                  'PT_ENROLL', 'FT_ENROLL', 'TOT_ENROLL', 'HOUSING', 'DORM_CAP', 'TOT_EMP',
                                  'SHELTER_ID','VAL_DATE'])
colleges_usa_small['CITY'] = colleges_usa_small['CITY'].str.title()
colleges_usa_small

,NAME,CITY
0,BRYANT & STRATTON COLLEGE-AMHERST,Getzville
1,CUNY SCHOOL OF LAW,Long Island City
2,ROCKLAND COMMUNITY COLLEGE,Suffern
3,SOUTHERN WESTCHESTER BOCES-PRACTICAL NURSING P...,Harrison
4,OHIO VALLEY COLLEGE OF TECHNOLOGY,East Liverpool
...,...,...
6852,ST CHARLES COMMUNITY COLLEGE,Cottleville
6853,RIDER UNIVERSITY,Lawrenceville
6854,THE COLLEGE OF NEW JERSEY,Ewing
6855,MADONNA UNIVERSITY,Livonia


In [21]:
df = pd.merge(df, cost_of_living, on=['City'])

In [22]:
# count how many colleges in each city (presence of many colleges in an area may be desirable)
df['Number of Colleges'] = df.apply(
    lambda row: len(colleges_usa_small.loc[colleges_usa_small['CITY'] == row['City']].index),
    axis=1)
df

,City,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,lat,lng,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Number of Colleges
0,Caracas,65.18,11.25,44.44,83.45,8.61,85.70,10.480594,-66.903606,1.80,9.12,11.97,0.07,1634.42,527.89,0
1,Johannesburg,84.08,53.99,59.98,47.39,51.26,83.93,-26.204103,28.047305,1.16,3.46,3.53,0.84,705.63,1167.89,0
2,Fortaleza,80.17,52.28,45.46,66.32,36.68,78.65,-3.732714,-38.526998,1.46,5.20,5.85,0.88,650.26,1307.54,0
3,Saint Louis,85.25,80.40,77.29,31.33,87.51,78.13,38.627003,-90.199404,1.96,6.54,6.54,0.61,980.65,2144.36,22
4,Mexico City,75.07,24.28,61.76,18.95,27.91,77.86,19.432608,-99.133208,1.60,3.20,7.47,0.56,747.10,587.01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Dusseldorf,83.45,72.52,75.98,8.61,88.02,15.34,51.227741,6.773456,2.13,8.31,4.47,1.28,1193.48,2301.70,0
212,Tokyo,80.00,47.57,71.53,30.54,69.29,13.91,35.676192,139.650311,2.30,11.80,8.52,0.98,1967.31,2065.67,0
213,Abu Dhabi,86.40,68.03,48.02,53.42,80.80,10.86,24.453884,54.377344,2.67,6.23,13.73,0.30,1779.93,2135.92,0
214,Marbella,81.96,60.14,83.64,32.27,75.24,9.11,36.510071,-4.882447,1.70,5.88,4.48,1.17,852.48,1619.72,0


In [25]:
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [28]:
# This is our Colab interactive version 
# (our web app version faced many technical difficulties)

#@title Rate importance of each of the following factors


movehub_rating = "None" #@param ["None", "Low", "Med", "High"]
purchase_power = "None" #@param ["None", "Low", "Med", "High"]
health_care = "None" #@param ["None", "Low", "Med", "High"]
quality_of_life = "None" #@param ["None", "Low", "Med", "High"]
pollution = "None" #@param ["None", "Low", "Med", "High"]
crime_rating = "None" #@param ["None", "Low", "Med", "High"]
number_of_colleges = "High" #@param ["None", "Low", "Med", "High"]

#@title Rate importance of the cost of each of the following items

cappucino = "None" #@param ["None", "Low", "Med", "High"]
cinema = "None" #@param ["None", "Low", "Med", "High"]
wine = "None" #@param ["None", "Low", "Med", "High"]
gasoline = "None" #@param ["None", "Low", "Med", "High"]
average_rent = "None" #@param ["None", "Low", "Med", "High"]
average_disposable_income = "None" #@param ["None", "Low", "Med", "High"]

features = ['Movehub Rating', 'Purchase Power', 'Health Care',
            'Pollution', 'Quality of Life', 'Crime Rating',
            'Cappuccino', 'Cinema', 'Wine', 'Gasoline', 'Avg Rent',
            'Avg Disposable Income', 'Number of Colleges']

weights = [
  movehub_rating,
  purchase_power,
  health_care,
  quality_of_life,
  pollution,
  crime_rating,
  number_of_colleges,
  cappucino,
  cinema,
  wine,
  gasoline,
  average_rent,
  average_disposable_income,
]
replace = {'0':0, 'None': 0, 'Low': 1, 'Med': 2, 'High': 3}
random_state = 3377
weights = np.array([replace[x] for x in weights])
weights *= [1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1]

X = df.loc[:][features]  # (n_samples, n_features)

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X) 

df_ = df.copy()
df_scaled = X.copy()
df_scaled.iloc[:, :] = scaled_X*weights

kmeans_per_k = KMeans(n_clusters=10, random_state=random_state).fit(scaled_X)
df_scaled['Score'] = df_['Score'] = kmeans_per_k.predict(scaled_X)

avg = df_scaled.groupby("Score").mean()[features].mean(axis=1)
df_['Score'] = df_['Score'].replace(avg.sort_values(ascending=True).index, range(len(avg)))

fig = px.scatter_mapbox(df_.sort_values('Score', ascending=False).round(),
                        lat="lat", lon="lng", color="Score", hover_name="City",
                        hover_data=features,
                        color_continuous_scale=px.colors.diverging.Spectral, size_max=15, zoom=1,
                        mapbox_style="carto-positron")
fig.show()
df_.sort_values('Score', ascending=False)[['City', 'Score'] + features].round()

,City,Score,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Number of Colleges
205,Hong Kong,9,86.0,50.0,68.0,15.0,60.0,16.0,3.0,6.0,10.0,2.0,5052.0,2210.0,0
194,Trondheim,9,80.0,51.0,82.0,15.0,83.0,20.0,4.0,12.0,13.0,2.0,1681.0,2801.0,0
60,Perth,9,95.0,62.0,81.0,24.0,75.0,50.0,3.0,11.0,10.0,1.0,2083.0,3359.0,0
142,Sydney,9,95.0,55.0,71.0,18.0,74.0,33.0,2.0,12.0,11.0,1.0,2789.0,2755.0,0
203,Bergen,9,81.0,51.0,85.0,18.0,84.0,17.0,4.0,12.0,12.0,2.0,1725.0,3003.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Chennai,0,78.0,33.0,67.0,78.0,44.0,33.0,1.0,2.0,5.0,1.0,302.0,302.0,0
137,Bucharest,0,72.0,23.0,52.0,75.0,29.0,33.0,1.0,5.0,3.0,1.0,509.0,383.0,0
134,Budapest,0,74.0,25.0,60.0,68.0,47.0,34.0,1.0,4.0,3.0,1.0,342.0,415.0,0
131,Mumbai,0,77.0,29.0,58.0,70.0,26.0,35.0,1.0,3.0,6.0,1.0,603.0,338.0,0


[Maps with express](https://plotly.com/python/plotly-express/#maps)

In [ ]:
# install these things to be able to use fig.write_image
!pip install plotly>=4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

# install this in order to connect to anvil
!pip install anvil-uplink

In [ ]:
# Functions to connect to Anvil (the frontend)
import anvil.server
anvil.server.connect("V42GBCPMCJBLXTXFCN3BZHBW-VZAABWBOI7U7HQJK")

# Test function
@anvil.server.callable
def say_hello():
  return("Hello world!")

# Use the K-means clustering algorithm to group cities with similar scores
@anvil.server.callable
def calculate_rankings(weights):
    replace = {'0':0, 'None': 0, 'Low': 1, 'Med': 2, 'High': 3}
    random_state = 3377
    weights = np.array([replace[x] for x in weights])
    weights *= [1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1]

    X = df.loc[:][features]  # (n_samples, n_features)

    scaler = StandardScaler()
    scaled_X = scaler.fit_transform(X) 
    
    df_ = df.copy()
    df_scaled = X.copy()
    df_scaled.iloc[:, :] = scaled_X*weights

    kmeans_per_k = KMeans(n_clusters=10, random_state=random_state).fit(scaled_X)
    df_scaled['Score'] = df_['Score'] = kmeans_per_k.predict(scaled_X)

    avg = df_scaled.groupby("Score").mean()[features].mean(axis=1)
    df_['Score'] = df_['Score'].replace(avg.sort_values(ascending=True).index, range(len(avg)))

    fig = px.scatter_mapbox(df_.sort_values('Score', ascending=False).round(),
                            lat="lat", lon="lng", color="Score", hover_name="City",
                            hover_data=features,
                            color_continuous_scale=px.colors.diverging.Spectral, size_max=15, zoom=1,
                            mapbox_style="carto-positron")

    fig.write_html("fig1.html")
    plot_html = open("fig1.html", "r")
    return plot_html.read()

anvil.server.wait_forever()